In [1]:
import pandas as pd
import csv
import numpy as np
from pprint import pprint

import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn import linear_model
from sklearn.linear_model import  LassoCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV, train_test_split

df_2012_train = pd.read_csv('df_2012_train.csv') 
df_2012_train = df_2012_train.set_index('fips')
df_2012_test = pd.read_csv('df_2012_test.csv') 
df_2012_test = df_2012_test.set_index('fips')
df_2016_train = pd.read_csv('df_2016_train.csv') 
df_2016_train = df_2016_train.set_index('county_name')
df_2016_test = pd.read_csv('df_2016_test.csv') 
df_2016_test = df_2016_test.set_index('county_name')

In [ ]:
df_2012_train.head(2)

In [2]:
df_2012_train_x = df_2012_train.drop(columns = ['state_abbreviation', 'county_name', 'diff_2012'])
df_2012_train_y = df_2012_train[['diff_2012']]

df_2016_train_x = df_2016_train.drop(columns = ['state_abbreviation', 'fips', 'diff_2016'])
df_2016_train_y = df_2016_train[['diff_2016']]

df_2012_test_x = df_2012_test.drop(columns = ['state_abbreviation', 'county_name', 'diff_2012'])
df_2012_test_y = df_2012_test[['diff_2012']]

df_2016_test_x = df_2016_test.drop(columns = ['state_abbreviation', 'fips', 'diff_2016'])
df_2016_test_y = df_2016_test[['diff_2016']]

full_train_x = [df_2012_train_x, df_2016_train_x]
full_train_x = pd.concat(full_train_x)
df_2012_train_y.rename(columns={'diff_2012':'diff'}, inplace=True)
df_2016_train_y.rename(columns={'diff_2016':'diff'}, inplace=True)

full_train_y = [df_2012_train_y, df_2016_train_y]
full_train_y = pd.concat(full_train_y)

full_test_x = [df_2012_test_x, df_2016_test_x]
full_test_x = pd.concat(full_test_x)
df_2012_test_y.rename(columns={'diff_2012':'diff'}, inplace=True)
df_2016_test_y.rename(columns={'diff_2016':'diff'}, inplace=True)

full_test_y = [df_2012_test_y, df_2016_test_y]
full_test_y = pd.concat(full_test_y)

C:\Users\sajor\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [3]:
#2012 FEATURE SELECTION

from sklearn.feature_selection import RFE, VarianceThreshold, SelectFromModel

estimator = LassoCV()

featureSelection = SelectFromModel(estimator, prefit=False, threshold='mean', max_features=None)
featureSelection.fit(df_2012_train_x, df_2012_train_y)

selectedFeatures = featureSelection.transform(df_2012_train_x)
#selectedFeatures

fs = df_2012_train_x.columns[featureSelection.get_support()]
#remaining features
keep = []
for i in range(len(fs)):
    keep.append(fs[i])
print(keep)

df_2012_train_x = df_2012_train_x[keep]
df_2012_test_x = df_2012_test_x[keep]

fts_2012 = keep


C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


['population', 'perc_female', 'perc_o18', 'perc_o65', 'perc_white_not_hisp', 'median_oo_value', 'monthly_costs_m', 'monthly_costs_nm', 'avg_hh_size', 'perc_1yr_res', 'perc_f_emp_2065', 'median_income', 'perc_pov', 'num_establishments']


In [4]:
len(keep)

14

In [5]:
#2016 feature selection

estimator = LassoCV()

featureSelection = SelectFromModel(estimator, prefit=False, threshold='mean', max_features=None)
featureSelection.fit(df_2016_train_x, df_2016_train_y)

selectedFeatures = featureSelection.transform(df_2016_train_x)
#selectedFeatures

fs = df_2016_train_x.columns[featureSelection.get_support()]
#remaining features
keep = []
for i in range(len(fs)):
    keep.append(fs[i])
print(keep)

df_2016_train_x = df_2016_train_x[keep]
df_2016_test_x = df_2016_test_x[keep]

fts_2016 = keep

C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


['population', 'perc_o18', 'perc_o65', 'perc_white', 'perc_hisp', 'perc_white_not_hisp', 'median_oo_value', 'monthly_costs_nm', 'avg_hh_size', 'perc_hs', 'hc_value_1k', 'median_income', 'perc_pov', 'num_establishments', 'annual_payroll']


In [6]:
estimator = LassoCV()

featureSelection = SelectFromModel(estimator, prefit=False, threshold='mean', max_features=None)
featureSelection.fit(full_train_x, full_train_y)

selectedFeatures = featureSelection.transform(full_train_x)
#selectedFeatures

fs = full_train_x.columns[featureSelection.get_support()]
#remaining features
keep = []
for i in range(len(fs)):
    keep.append(fs[i])
print(keep)

full_train_x = full_train_x[keep]
full_test_x = full_test_x[keep]

fts_full = keep

C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


['population', 'perc_o18', 'perc_o65', 'perc_white', 'perc_hisp', 'perc_white_not_hisp', 'median_oo_value', 'monthly_costs_nm', 'avg_hh_size', 'perc_1yr_res', 'perc_f_lang', 'perc_hs', 'perc_f_emp_2065', 'wholesale_value_1k', 'hc_value_1k', 'median_income', 'perc_pov', 'num_establishments']


In [7]:
ls = LassoCV(random_state = 33)
pprint(ls.get_params())

{'alphas': None,
 'copy_X': True,
 'cv': 'warn',
 'eps': 0.001,
 'fit_intercept': True,
 'max_iter': 1000,
 'n_alphas': 100,
 'n_jobs': None,
 'normalize': False,
 'positive': False,
 'precompute': 'auto',
 'random_state': 33,
 'selection': 'cyclic',
 'tol': 0.0001,
 'verbose': False}


In [ ]:
from sklearn.model_selection import RandomizedSearchCV


eps = [0.0001, 0.001, 0.01]
max_iter = [int(x) for x in np.linspace(start = 10, stop = 2000, num = 10)]
alpha = [0.5, 1, 1.5]

# Create the random grid
random_grid = {
               'max_iter': max_iter,
               'alpha': alpha}
pprint(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
ls = Lasso()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
ls_random = RandomizedSearchCV(estimator = ls, param_distributions = random_grid, n_iter = 50, cv = 3, verbose=2, random_state=31, n_jobs = -1)
# Fit the random search model
ls_random.fit(df_2012_train_x, df_2012_train_y)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score

def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    mse = mean_squared_error(test_labels, predictions)
    rmse = np.sqrt(mse)
    print("rmse:",rmse)
    r2 = explained_variance_score(test_labels, predictions)
    print("r2:",r2)

print("BASE MODEL")
base_model = Lasso(random_state = 31)
base_model.fit(df_2012_train_x, df_2012_train_y)
base_accuracy = evaluate(base_model, df_2012_test_x, df_2012_test_y)

In [ ]:
ls_random.best_params_

In [ ]:
print('best random')
best_random =ls_random.best_estimator_
random_accuracy = evaluate(best_random, df_2012_test_x, df_2012_test_y)

In [8]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
max_iter = [int(x) for x in np.linspace(start = 10, stop = 2000, num = 20)]
n_alphas = [int(x) for x in np.linspace(start = 10, stop = 500, num = 20)]

param_grid = {'max_iter': max_iter,
               'n_alphas': n_alphas,
             'eps': [0.0001]}
# Create a based model
ls = LassoCV()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = ls, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [9]:
# Fit the grid search to the data
grid_search.fit(df_2012_train_x, df_2012_train_y)
grid_search.best_params_



Fitting 3 folds for each of 400 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   14.6s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   35.6s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed:  3.3min finished
C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNIN

{'eps': 0.0001, 'max_iter': 219, 'n_alphas': 35}

In [10]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    mse = mean_squared_error(test_labels, predictions)
    rmse = np.sqrt(mse)
    print("mse:", mse)
    print("rmse:",rmse)
    mae = mean_absolute_error(test_labels, predictions)
    print("mae:", mae)
    r2 = explained_variance_score(test_labels, predictions)
    print("r2:",r2)

In [11]:
df_2012_train_x2 = df_2012_train.drop(columns = ['state_abbreviation', 'county_name', 'diff_2012'])
df_2012_train_y2 = df_2012_train[['diff_2012']]

df_2016_train_x2 = df_2016_train.drop(columns = ['state_abbreviation', 'fips', 'diff_2016'])
df_2016_train_y2 = df_2016_train[['diff_2016']]

df_2012_test_x2 = df_2012_test.drop(columns = ['state_abbreviation', 'county_name', 'diff_2012'])
df_2012_test_y2 = df_2012_test[['diff_2012']]

df_2016_test_x2 = df_2016_test.drop(columns = ['state_abbreviation', 'fips', 'diff_2016'])
df_2016_test_y2 = df_2016_test[['diff_2016']]

full_train_x2 = [df_2012_train_x2, df_2016_train_x2]
full_train_x2 = pd.concat(full_train_x2)
df_2012_train_y2.rename(columns={'diff_2012':'diff'}, inplace=True)
df_2016_train_y2.rename(columns={'diff_2016':'diff'}, inplace=True)

full_train_y2 = [df_2012_train_y2, df_2016_train_y2]
full_train_y2 = pd.concat(full_train_y2)

full_test_x2 = [df_2012_test_x2, df_2016_test_x2]
full_test_x2 = pd.concat(full_test_x2)
df_2012_test_y2.rename(columns={'diff_2012':'diff'}, inplace=True)
df_2016_test_y2.rename(columns={'diff_2016':'diff'}, inplace=True)

full_test_y2 = [df_2012_test_y2, df_2016_test_y2]
full_test_y2 = pd.concat(full_test_y2)

C:\Users\sajor\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [12]:
#fts_2012 = ['population', 'perc_female', 'perc_o18', 'perc_o65', 'perc_white_not_hisp', 'median_oo_value', 'monthly_costs_m', 'monthly_costs_nm', 'avg_hh_size', 'perc_1yr_res', 'perc_f_emp_2065', 'median_income', 'perc_pov', 'num_establishments']
test_x = df_2016_test_x2[fts_2012]
full_x = full_test_x2[fts_2012]



print('2012 GRID Search on 2012')
grid_accuracy = evaluate(grid_search, df_2012_test_x, df_2012_test_y)
print('2012 GRID Search on 2016')
grid_accuracy = evaluate(grid_search, test_x, df_2016_test_y2)
print('2012 GRID Search on Full')
grid_accuracy = evaluate(grid_search, full_x, full_test_y2)

2012 GRID Search on 2012
mse: 0.04893156329849512
rmse: 0.22120479944724328
mae: 0.15636532891376098
r2: 0.41994606183834593
2012 GRID Search on 2016
mse: 1.0789570768363113
rmse: 1.038728586704107
mae: 1.0045918043439548
r2: 0.15442632634195474
2012 GRID Search on Full
mse: 0.5639443200674032
rmse: 0.7509622627452083
mae: 0.580478566628858
r2: -2.384233526310513


In [13]:
grid_search.best_params_

{'eps': 0.0001, 'max_iter': 219, 'n_alphas': 35}

In [14]:
#fts = ['population', 'perc_o18', 'perc_white', 'perc_black', 'perc_white_not_hisp', 'perc_oo', 'monthly_costs_m', 'monthly_costs_nm', 'num_nonemployers', 'man_value_1k', 'wholesale_value_1k', 'num_establishments', 'annual_payroll', 'min_num_firms']
list(zip(fts_2012,list(grid_search.best_estimator_.coef_)))

[('population', 1.3777881878817269),
 ('perc_female', 0.5962361759413386),
 ('perc_o18', 1.004887818744711),
 ('perc_o65', -1.3337477598583043),
 ('perc_white_not_hisp', -1.6088999260721648),
 ('median_oo_value', 0.7075715640123761),
 ('monthly_costs_m', -0.9980402512881635),
 ('monthly_costs_nm', 2.804701687026948),
 ('avg_hh_size', -0.5365222266935501),
 ('perc_1yr_res', 0.7920258831229773),
 ('perc_f_emp_2065', 0.7950917698871733),
 ('median_income', -0.7616569595035113),
 ('perc_pov', 0.3549888279723641),
 ('num_establishments', -1.1799239401220425)]

In [15]:


# Fit the grid search to the data
grid_search.fit(df_2016_train_x, df_2016_train_y)
grid_search.best_params_


Fitting 3 folds for each of 400 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done 198 tasks      | elapsed:   29.8s
[Parallel(n_jobs=-1)]: Done 401 tasks      | elapsed:   59.8s
[Parallel(n_jobs=-1)]: Done 684 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 1049 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed:  2.8min finished
C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNIN

{'eps': 0.0001, 'max_iter': 114, 'n_alphas': 35}

In [16]:
#fts_2016 = ['population', 'perc_o18', 'perc_o65', 'perc_white', 'perc_hisp', 'perc_white_not_hisp', 'median_oo_value', 'monthly_costs_nm', 'avg_hh_size', 'perc_hs', 'hc_value_1k', 'median_income', 'perc_pov', 'num_establishments', 'annual_payroll']
test_x = df_2012_test_x2[fts_2016]
full_x = full_test_x2[fts_2016]

print('2016 GRID Search on 2012')
grid_accuracy = evaluate(grid_search, test_x, df_2012_test_y2)
print('2016 GRID Search on 2016')
grid_accuracy = evaluate(grid_search, df_2016_test_x, df_2016_test_y)
print('2016 GRID Search on Full')
grid_accuracy = evaluate(grid_search, full_x, full_test_y2)

2016 GRID Search on 2012
mse: 0.07254884313615793
rmse: 0.26934892451271814
mae: 0.22380370826147492
r2: 0.5522769156849741
2016 GRID Search on 2016
mse: 0.02461041986152264
rmse: 0.15687708520215002
mae: 0.12431716460901379
r2: 0.7322261588317815
2016 GRID Search on Full
mse: 0.04857963149884029
rmse: 0.2204078753103897
mae: 0.17406043643524435
r2: 0.555139672258397


In [ ]:
grid_search.best_params_

In [17]:
list(zip(fts_2016,list(grid_search.best_estimator_.coef_)))

[('population', 0.25130663184845026),
 ('perc_o18', 0.3402272006768526),
 ('perc_o65', -0.33952507415054156),
 ('perc_white', -0.09943200542060257),
 ('perc_hisp', -0.28062635414336295),
 ('perc_white_not_hisp', -2.5322037624780775),
 ('median_oo_value', 0.7279065850518883),
 ('monthly_costs_nm', 0.7717686219617513),
 ('avg_hh_size', -0.3968131570454854),
 ('perc_hs', 0.5184082945916201),
 ('hc_value_1k', -0.05739876090996707),
 ('median_income', -0.46136253824338336),
 ('perc_pov', 0.22219495518904642),
 ('num_establishments', -0.5759982914089486),
 ('annual_payroll', 0.6525068795421799)]

In [18]:
# Fit the grid search to the data
grid_search.fit(full_train_x, full_train_y)
grid_search.best_params_

Fitting 3 folds for each of 400 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done 174 tasks      | elapsed:   32.1s
[Parallel(n_jobs=-1)]: Done 377 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 660 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1025 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed:  3.9min finished
C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNIN

{'eps': 0.0001, 'max_iter': 114, 'n_alphas': 61}

In [19]:
#fts_full = ['population', 'perc_o18', 'perc_o65', 'perc_white', 'perc_hisp', 'perc_white_not_hisp', 'median_oo_value', 'monthly_costs_nm', 'avg_hh_size', 'perc_1yr_res', 'perc_f_lang', 'perc_hs', 'perc_f_emp_2065', 'wholesale_value_1k', 'hc_value_1k', 'median_income', 'perc_pov', 'num_establishments']
test_x = df_2012_test_x2[fts_full]
test2_x = df_2016_test_x2[fts_full]

print('FULL GRID Search on 2012')
grid_accuracy = evaluate(grid_search, test_x, df_2012_test_y2)
print('FULL GRID Search on 2016')
grid_accuracy = evaluate(grid_search, test2_x, df_2016_test_y2)
print('FULL GRID Search on Full')
grid_accuracy = evaluate(grid_search, full_test_x, full_test_y)

FULL GRID Search on 2012
mse: 0.03397491409174444
rmse: 0.184322852874364
mae: 0.14739565457176873
r2: 0.595915387311658
FULL GRID Search on 2016
mse: 0.025011660335829702
rmse: 0.15815075192938446
mae: 0.12454746033913967
r2: 0.7279882683070892
FULL GRID Search on Full
mse: 0.02949328721378707
rmse: 0.17173609758518177
mae: 0.13597155745545422
r2: 0.6749296595023925


In [ ]:
grid_search.best_params_

In [20]:
list(zip(fts_full,list(grid_search.best_estimator_.coef_)))

[('population', 0.7695820692923028),
 ('perc_o18', 0.6602851952647957),
 ('perc_o65', -0.7237517364590067),
 ('perc_white', -0.061850532786815605),
 ('perc_hisp', -0.4462369967039323),
 ('perc_white_not_hisp', -2.2184186391050766),
 ('median_oo_value', 0.6049309562750149),
 ('monthly_costs_nm', 0.5682077275298176),
 ('avg_hh_size', -0.3584701548016614),
 ('perc_1yr_res', 0.4720521808579215),
 ('perc_f_lang', 0.3293471759681122),
 ('perc_hs', 0.46451950925723395),
 ('perc_f_emp_2065', 0.44200977153089216),
 ('wholesale_value_1k', -0.2967753772574962),
 ('hc_value_1k', 0.1887418986082729),
 ('median_income', -0.5122548469087228),
 ('perc_pov', 0.3387523234431195),
 ('num_establishments', -0.4182786440675589)]